## 불어를 영어로 번역하도록 가르쳐 봅시다.


하나의 시퀀스를 다른 시퀀스로 바꾸는 두개의 RNN이 함께 동작하는 seq2seq network의 아이디어를 사용해 봅시다. 인코더 네트워크는 입력 시퀀스를 벡터로 압축하고, 디코더 네트워크는 해당 벡터를 새로운 시퀀스로 펼칩니다.
<br>

Attention Mechanism은 디코더가 입력 시퀀스의 범위에 초점을 맞춰야합니다.

``` 
> input, 
= target, 
< output
    
> il est en train de peindre un tableau
= he is painting a picture
< he is painting a picture
```



## Word Representation

- word2index
- unknown word

네트워크의 input 과 target으로 넣기 위해 단어당 고유 index가 필요합니다. 따라서 word2index 그리고 index2word를 가지게 있어야합니다. 그리고 희귀단어를 대체하는데 사용할 단어의 빈도 word2count를 가진 Lang 이라는 헬퍼 클래스를 사용해 봅시다.

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

In [3]:
SOS_token =0
EOS_token =1

In [35]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0:"SOS", 1:"EOS"}
        self.n_words = 2 #SOS 와 EOS를 포함한다.
    
    # 문장이 들어온다면, 단어로 쪼갠뒤 addWord로 넣어줍니다.
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    # word를 key에 index를 넣어준다. 그리고 count 값도 넣어줍니다.
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [28]:
# 유니코드 문자를 ASCII 로 변환하고 모든 문자를 소문자로 만들고, 구두점을 지워줍니다.
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mm'
    )

In [29]:
# 소문자, normalize, 문자아닌 문자 제거
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [30]:
# 데이터 파일을 line으로 읽은 다음에 pair로 자료형을 구성합니다. 혹시 뒤집힐수 있으니 reverse 플래그를 추가합니다.
def readLangs(lang1, lang2, reverse = False):
    print("Reading lines...")
    
    # 파일을 읽고 줄로 분리
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # 모든 줄을 쌍으로 분리하고 정규화
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    
    # 쌍을 뒤집고, Lang 인스턴스 생성
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
        
    return input_lang, output_lang, pairs

In [31]:
# 데이터 셋의 최대길이를 10단어로 만들고, prefixes들을 i am , he is 등으로 만들어봅시다.
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [36]:
# 텍스트 파일을 읽고, line으로 분리하고, line에서 Piar를 만들어 냅니다. + normalize + pair 에서 단어 list 생성

def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 9874 sentence pairs
Counting words...
Counted words:
fra 3802
eng 2689
['nous sommes normales .', 'we re normal .']


## SEQ2SEQ

RNN은 시퀀스에서 작동하고, 후속단계의 입력으로 자신의 출력을 사용하는 네트워크입니다.
- Seq2Seq : 인코더와 디코더라는 두개의 RNN으로 구성된 모델입니다.시퀀스의 길이와 순서가 자유롭습니다.
- Encoder : 입력 시퀀스를 읽고 단일 vector를 출력합니다.
- Decoder : 해당 백터를 읽어 출력 시퀀스를 구성합니다.

In [38]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [49]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        # input이 embedding으로 들어오면
        self.embedding = nn.Embedding(input_size, hidden_size)
        # 이전 히든과, embedded 된 hidden을 받아서
        self.gru = nn.GRU(hidden_size, hidden_size)
        # gru 는 output과 hidden 을 배출한다.
        
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1,1,-1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1,1, self.hidden_size, device=device)

In [50]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        slef.softmax = nn.LogSoftmax(dim = 1)
        
    def forward(self, input, hidden):
        output = self.embedding(input).view(1,1,-1)
        output = F.relu(output)
        output, hidden = self.gru(output,hidden)
        ouput = self.softmax(self.out(output[0]))
        return output, hidden
    
    def initHidden(self):
        return torch.zeors(1,1,self.hidden_size, device= device)

## Attention decoder

문맥 벡터만 인코더와 디코더 사이로 전달 된다면, 단일 벡터가 전체 문장을 인코딩해야한다는 부담을 가지게 됩니다. 어텐션은 디코더 네트워크가 자기 출력의 모든 단계에서 인코더 출력의 부분에 집중하게 해줍니다. 
- attention weight : 집중할 location 정보입니다. 인코더의 output과 곱해집니다.

### torch.bmm 함수 batch matrix multiplication(bmm, 배치 행렬곱)

2개 이상의 차원을 지닌 텐서가 주어졌을 때 뒤 2개 차원에 대해서는 행렬곱을 수행하고 앞의 다른 차원은 미니배치로 취급합니다. 따라서 앞의 차원들은 크기가 같아야 하고, 뒤 2개 차원은 행렬 곱을 수행하기 위한 적절한 크기를 지녀야 합니다.

In [51]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p = 0.1, max_length = MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size*2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size*2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1,1,-1)
        embedded = self.dropout(embedded)
        
        attn_weight = F.softmax(
            self.attn(torch.cat((embedded[0],hidden[0]),1 )), dim=1)
        attn_applied = torch.bmm(attn_weight.unsqueeze(0),
                                encoder_outputs.unsqueeze(0))
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weight
    
    def initHidden(self):
        return torch.zeros(1,1,self.hidden_size, device=device)

In [52]:
# 학습을 위해서, 각 pair 마다 tensor(입력문장의 단어주소와) Tensor(Target의 단어주소)
# 이 벡터를 생성하는 동안 시퀀스에 EOS 토큰을 추가합시다.

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1,1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)